In [10]:
import glob
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [12]:
def load_data(label_type, data_type):
    path = f'./{label_type.lower()}/*/*.tif'
    labels = []
    images = []
    for file in glob.glob(path):
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            if label_type.lower() == "lof":
                labels.append(0)
            elif label_type.lower() == "gas":
                labels.append(1)
    return np.array(images), np.array(labels)


# Load and preprocess the data
lof_images, lof_labels = load_data('LOF', 'train')
gas_images, gas_labels = load_data('GAS', 'train')

images = np.concatenate([lof_images, gas_images])
labels = np.concatenate([lof_labels, gas_labels])

In [24]:
# Option 1: Increase the size of your input images
for i in range(len(images)):
    images[i] = cv2.resize(images[i], (128, 128))  # for example

images = np.expand_dims(images, axis=-1)

In [27]:
# Define the model
model = Sequential()

# Option 2: Decrease the size of your kernel
model.add(Conv2D(32, (2, 2), activation='relu',
          input_shape=(images.shape[1], images.shape[2], 1)))

# Option 3: Increase the padding
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [28]:
# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(images, labels, epochs=10)

# Evaluate the model on the training data
train_loss, train_acc = model.evaluate(images, labels)
print(f'Training accuracy: {train_acc}')

Epoch 1/10


ValueError: Exception encountered when calling Conv2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_8_1/conv2d_8_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](data, sequential_8_1/conv2d_8_1/convolution/ReadVariableOp)' with input shapes: [32,1,1,1], [2,2,1,32].[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(32, 1, 1, 1), dtype=float32)